
# IBM Applied Data Science Capstone Course by Coursera
## Final Report
### Opening a pub in Milan, Italy

#### Build a dataframe of neighborhoods in Milan by web scraping the data from Wikipedia page
#### Get the geographical coordinates of the neighborhoods
#### Obtain the venue data for the neighborhoods from Foursquare API
#### Explore and cluster the neighborhoods
#### Select the best cluster to open a new pub

### 1. Import libraries¶


In [10]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geopy
!pip install geocoder


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium

import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 94 kB 2.1 MB/s eta 0:00:011
Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame¶


In [11]:
# send the GET request
data = requests.get("https://it.wikipedia.org/wiki/Categoria:Quartieri_di_Milano").text

In [12]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [13]:
# create a list to store neighborhood data
neighborhoodList = []

In [14]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [15]:
# create a new DataFrame from the list
milan_df = pd.DataFrame({"Neighborhood": neighborhoodList})

milan_df.head()

,Neighborhood
0,Quartieri di Milano
1,Affori
2,Arzaga
3,Baggio (Milano)
4,Barona (Milano)


In [16]:
# print the number of rows of the dataframe
milan_df.shape

(94, 1)

### 3. Get latitudes and longitudes


In [18]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Milan, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [21]:
# call the function to get the coordinates, store in a new list using list comprehension
coordinates = [ get_latlng(neighborhood) for neighborhood in milan_df["Neighborhood"].tolist() ]

In [22]:
coordinates

[[45.47139000000004, 9.206450000000075],
 [45.51410000000004, 9.173530000000028],
 [45.45467000000008, 9.136130000000037],
 [45.46324000000004, 9.092700000000036],
 [45.433710000000076, 9.15160000000003],
 [45.52149000000003, 9.213260000000048],
 [45.49178000000006, 9.14187000000004],
 [45.57643865577999, 9.061037195982879],
 [45.46796000000006, 9.18178000000006],
 [45.503130000000056, 9.161220000000071],
 [45.515550000000076, 9.150940000000048],
 [45.52825000000007, 9.180710000000033],
 [45.492380000000026, 9.15187000000003],
 [45.456180000000074, 9.224880000000041],
 [45.52688505059429, 8.907860097898094],
 [45.54799234966403, 8.994257200554808],
 [45.48873000000003, 9.22986000000003],
 [45.46692000000007, 9.24147000000005],
 [45.50198373621629, 9.264641249871804],
 [45.415680000000066, 9.239790000000028],
 [45.41668431692717, 9.164223329947587],
 [45.500860000000046, 9.265130000000056],
 [45.503460000000075, 9.248800000000074],
 [45.47708000000006, 9.226600000000076],
 [45.526310000

In [23]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
coordinates_df = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [24]:
# merge the coordinates into the original dataframe
milan_df['Latitude'] = coordinates_df['Latitude']
milan_df['Longitude'] = coordinates_df['Longitude']

In [25]:
# check the neighborhoods and the coordinates
print(milan_df.shape)
milan_df

(94, 3)


,Neighborhood,Latitude,Longitude
0,Quartieri di Milano,45.471390,9.206450
1,Affori,45.514100,9.173530
2,Arzaga,45.454670,9.136130
3,Baggio (Milano),45.463240,9.092700
4,Barona (Milano),45.433710,9.151600
5,Bicocca (Milano),45.521490,9.213260
6,Boldinasco,45.491780,9.141870
7,Borgo degli Ortolani,45.576439,9.061037
8,Bottonuto,45.467960,9.181780
9,Bovisa,45.503130,9.161220


In [26]:
# save the DataFrame as CSV file
milan_df.to_csv("milan_df.csv", index=False)

### 4. Create a map of Milan with neighborhoods superimposed on top¶

In [28]:
# get the coordinates of Milan
address = 'Milan, Italy'

geolocator = Nominatim(user_agent="http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Milan, Italy {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Milan, Italy 45.4668, 9.1905.


In [29]:
# create map of Toronto using latitude and longitude values
map_milan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(milan_df['Latitude'], milan_df['Longitude'], milan_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_milan)  
    
map_milan

In [30]:
# save the map as HTML file
map_milan.save('map_milan.html')

### 5. Use the Foursquare API to explore the neighborhoods¶

In [32]:
# define Foursquare Credentials and Version
CLIENT_ID = 'PP0TFYWOO3SO353TXJVQWAQYAUVT4X5BRII4GLEM5OOLTUQ2' # your Foursquare ID
CLIENT_SECRET = 'WP1DNQ2T5JTX4UGB20FIAHSELKA3BLTDN5GZE34M43NF5JBY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PP0TFYWOO3SO353TXJVQWAQYAUVT4X5BRII4GLEM5OOLTUQ2
CLIENT_SECRET:WP1DNQ2T5JTX4UGB20FIAHSELKA3BLTDN5GZE34M43NF5JBY


In [33]:
# Now, let's get the top 100 venues that are within a radius of 2 kms.

radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(milan_df['Latitude'], milan_df['Longitude'], milan_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [118]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7209, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Quartieri di Milano,45.47139,9.20645,Crosta,45.470739,9.208922,Pizza Place
1,Quartieri di Milano,45.47139,9.20645,Porta Venezia,45.474310,9.204898,Monument / Landmark
2,Quartieri di Milano,45.47139,9.20645,Chateau Monfort,45.468158,9.207319,Hotel
3,Quartieri di Milano,45.47139,9.20645,Villa Necchi Campiglio,45.468181,9.201570,Museum
4,Quartieri di Milano,45.47139,9.20645,Giardini Indro Montanelli,45.474395,9.201074,Park


In [119]:
venues_df.groupby(["Neighborhood"]).count()


,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Affori,57,57,57,57,57,57
Arzaga,100,100,100,100,100,100
Baggio (Milano),38,38,38,38,38,38
Barona (Milano),21,21,21,21,21,21
Bicocca (Milano),100,100,100,100,100,100
Boldinasco,100,100,100,100,100,100
Borgo degli Ortolani,83,83,83,83,83,83
Bottonuto,100,100,100,100,100,100
Bovisa,100,100,100,100,100,100


In [120]:
# print out the list of categories
venues_df['VenueCategory'].unique() #[:50]

array(['Pizza Place', 'Monument / Landmark', 'Hotel', 'Museum', 'Park',
       'Planetarium', 'Wine Bar', 'Art Gallery', 'Accessories Store',
       'Boutique', 'Coffee Shop', 'Bakery', 'Toy / Game Store', 'Café',
       "Men's Store", 'Dessert Shop', 'Cupcake Shop', 'Electronics Store',
       'Cocktail Bar', 'Italian Restaurant', 'Candy Store',
       'Ice Cream Shop', 'Hotel Bar', 'Plaza', 'Hostel', 'Art Museum',
       'Opera House', 'Scenic Lookout', 'Pastry Shop', 'Department Store',
       'College Arts Building', 'Chinese Restaurant', 'Gift Shop',
       'Supermarket', 'Restaurant', 'Sandwich Place', 'Sushi Restaurant',
       'Historic Site', 'Theater', 'Gourmet Shop', 'Salad Place',
       'Camera Store', 'Beer Bar', 'Kebab Restaurant', 'Pool Hall',
       'Ballroom', 'Health Food Store', 'Brewery',
       'Paper / Office Supplies Store', 'Gym / Fitness Center',
       'Hobby Shop', 'Soccer Field', 'Gym', 'Bistro',
       'Seafood Restaurant', 'Piadineria', 'Steakhouse', 'Pub

In [121]:
# check if the results contain "pub"
"Pub" in venues_df['VenueCategory'].unique()

True

##### ok, we have "pub" among the categories, but what about "Irish Pub" or "Gastropub", maybe it would be better to calculate a new field based on a string condition

In [122]:
venues_df['simplified_category'] = venues_df['VenueCategory'].apply(lambda x: 'Pub' if 'Pub' in x else x)
venues_df['simplified_category'] = venues_df['VenueCategory'].apply(lambda x: 'Pub' if 'Beer' in x else x)



In [125]:
venues_df.head(10)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory,simplified_category
0,Quartieri di Milano,45.47139,9.20645,Crosta,45.470739,9.208922,Pizza Place,Pizza Place
1,Quartieri di Milano,45.47139,9.20645,Porta Venezia,45.474310,9.204898,Monument / Landmark,Monument / Landmark
2,Quartieri di Milano,45.47139,9.20645,Chateau Monfort,45.468158,9.207319,Hotel,Hotel
3,Quartieri di Milano,45.47139,9.20645,Villa Necchi Campiglio,45.468181,9.201570,Museum,Museum
4,Quartieri di Milano,45.47139,9.20645,Giardini Indro Montanelli,45.474395,9.201074,Park,Park
5,Quartieri di Milano,45.47139,9.20645,"Civico Planetario ""Ulrico Hoepli""",45.473582,9.203954,Planetarium,Planetarium
6,Quartieri di Milano,45.47139,9.20645,Giardini di Villa Reale,45.472127,9.199721,Park,Park
7,Quartieri di Milano,45.47139,9.20645,Bicerin,45.476988,9.202432,Wine Bar,Wine Bar
8,Quartieri di Milano,45.47139,9.20645,GAM Galleria d'Arte Moderna di Milano,45.472683,9.199923,Art Gallery,Art Gallery
9,Quartieri di Milano,45.47139,9.20645,Dolce&Gabbana,45.468767,9.198626,Accessories Store,Accessories Store


### 6. Analyze Each Neighborhood¶

In [126]:
# one hot encoding
milan_onehot = pd.get_dummies(venues_df[['simplified_category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
milan_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [milan_onehot.columns[-1]] + list(milan_onehot.columns[:-1])
milan_onehot = milan_onehot[fixed_columns]

print(milan_onehot.shape)
milan_onehot.head()

(7209, 274)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comic Shop,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Women's Store
0,Quartieri di Milano,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [127]:
milan_grouped = milan_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(milan_grouped.shape)
milan_grouped

(94, 274)


,Neighborhoods,Abruzzo Restaurant,Accessories Store,Adult Education Center,African Restaurant,Agriturismo,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Ballroom,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Calabria Restaurant,Camera Store,Campanian Restaurant,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,Comedy Club,Comic Shop,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doner Restaurant,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Lombard Restaurant,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Opera House,Optical Shop,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Lab,Piadineria,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Puglia Restaurant,Racecourse,Racetrack,Radio Station,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Club,Roman Restaurant,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sardinian Restaurant,Sauna / Steam Room,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Sicilian Restaurant,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,TV Station,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Women's Store
0,Affori,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [128]:
### identify how many neighborhoods have pubs:
len(milan_grouped[milan_grouped["Pub"] > 0])

53

### Create a new DataFrame for Pubs data only

In [129]:



milan_pubs = milan_grouped[["Neighborhoods","Pub"]]
milan_pubs.head()

,Neighborhoods,Pub
0,Affori,0.017544
1,Arzaga,0.010000
2,Baggio (Milano),0.026316
3,Barona (Milano),0.000000
4,Bicocca (Milano),0.040000


### 7. Cluster Neighborhoods¶
#### Run k-means to cluster the neighborhoods in Milan into 3 clusters.

In [148]:
# set number of clusters
kclusters =3

milan_clustering =  milan_pubs.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 2, 0, 1, 0, 2, 0, 0, 0], dtype=int32)

In [149]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
milan_merged = milan_pubs.copy()

# add clustering labels
milan_merged["Cluster Labels"] = kmeans.labels_

In [150]:
milan_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
milan_merged.head()

,Neighborhood,Pub,Cluster Labels
0,Affori,0.017544,2
1,Arzaga,0.010000,0
2,Baggio (Milano),0.026316,2
3,Barona (Milano),0.000000,0
4,Bicocca (Milano),0.040000,1


In [151]:
# merge milan_grouped with milan data (milan_df) to add latitude/longitude for each neighborhood
milan_merged = milan_merged.join(milan_df.set_index("Neighborhood"), on="Neighborhood")

print(milan_merged.shape)
milan_merged.head() # check the last columns!

(94, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
0,Affori,0.017544,2,45.51410,9.17353
1,Arzaga,0.010000,0,45.45467,9.13613
2,Baggio (Milano),0.026316,2,45.46324,9.09270
3,Barona (Milano),0.000000,0,45.43371,9.15160
4,Bicocca (Milano),0.040000,1,45.52149,9.21326


In [152]:
# sort the results by Cluster Labels
print(milan_merged.shape)
milan_merged.sort_values(["Cluster Labels"], inplace=True)
milan_merged

(94, 5)


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
93,Zona Tortona,0.000000,0,45.452620,9.165320
28,Garegnano,0.000000,0,45.504690,9.136970
60,Quartiere Chiesa Rossa,0.000000,0,45.416684,9.164223
59,Quartiere Campo dei Fiori,0.000000,0,45.526885,8.907860
31,Gratosoglio,0.000000,0,45.414590,9.171220
32,Greco (Milano),0.010000,0,45.497020,9.212120
58,Quadrilatero della moda,0.000000,0,45.467960,9.181780
57,QT8,0.000000,0,45.486030,9.138410
35,Lampugnano,0.010000,0,45.491630,9.121960
56,Progetto Porta Nuova,0.010000,0,45.479710,9.192470


### And now... let's visualize the resulting clusters!

In [153]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milan_merged['Latitude'], milan_merged['Longitude'], milan_merged['Neighborhood'], milan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [154]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examining the clusters

#### Cluster 0

In [155]:
milan_merged.loc[milan_merged['Cluster Labels'] == 0]


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
93,Zona Tortona,0.00,0,45.452620,9.165320
28,Garegnano,0.00,0,45.504690,9.136970
60,Quartiere Chiesa Rossa,0.00,0,45.416684,9.164223
59,Quartiere Campo dei Fiori,0.00,0,45.526885,8.907860
31,Gratosoglio,0.00,0,45.414590,9.171220
32,Greco (Milano),0.01,0,45.497020,9.212120
58,Quadrilatero della moda,0.00,0,45.467960,9.181780
57,QT8,0.00,0,45.486030,9.138410
35,Lampugnano,0.01,0,45.491630,9.121960
56,Progetto Porta Nuova,0.01,0,45.479710,9.192470


#### Cluster 1

In [156]:
milan_merged.loc[milan_merged['Cluster Labels'] == 1]


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
71,Quinto Romano,0.046512,1,45.476840,9.088840
64,Quartiere Stadera,0.040000,1,45.434285,9.182215
46,Niguarda,0.050000,1,45.518400,9.192010
16,Centro direzionale di Milano,0.043478,1,45.501984,9.264641
15,Cavriano,0.050000,1,45.466920,9.241470
49,Ortica (Milano),0.060000,1,45.470960,9.241600
17,Chiaravalle (Milano),0.060606,1,45.415680,9.239790
18,Chinatown di Milano,0.046512,1,45.500860,9.265130
13,Cascina Merlata,0.100000,1,45.547992,8.994257
20,Città Studi,0.040000,1,45.477080,9.226600


#### Cluster 2

In [157]:
milan_merged.loc[milan_merged['Cluster Labels'] == 2]


,Neighborhood,Pub,Cluster Labels,Latitude,Longitude
6,Borgo degli Ortolani,0.012048,2,45.576439,9.061037
78,San Leonardo (Milano),0.020000,2,45.502130,9.100800
87,Turro (Milano),0.020000,2,45.494520,9.221710
82,Sella Nuova,0.012500,2,45.458678,9.108251
81,Segnano,0.020000,2,45.506910,9.206050
2,Baggio (Milano),0.026316,2,45.463240,9.092700
14,Casoretto,0.030000,2,45.488730,9.229860
89,Vialba,0.012500,2,45.514910,9.128150
90,Vigentino,0.020000,2,45.433720,9.201040
91,Villaggio dei Giornalisti,0.030000,2,45.495990,9.202340


### Final Observations


This exercise has shown a clear difference in the distribution of pubs in the city, with one cluster that appears to be crowded (cluster 1), one cluster mostly empty (cluster 0) and one cluster in the middle.

It would then be advisable to pick up a neighborhood in Cluster 2 as it appear to be the sweet spot between a crowded neighborhood and a, what we can suppose as a residential one.

However if the unique selling point and the proposition is strong then we can advise to pick up a quarter in Cluster 1, as the high number of pubs may indicate a more liveful neighborhood.